In [231]:
from sklearn.datasets import load_iris
import numpy as np

In [232]:
x, t = load_iris(return_X_y=True)

In [233]:
x.shape, t.shape

((150, 4), (150,))

In [234]:
print(type(x))
print(type(t))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [235]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [236]:
x = torch.tensor(x, dtype=torch.float32)
t = torch.tensor(t, dtype=torch.int64)
x, t

(tensor([[5.1000, 3.5000, 1.4000, 0.2000],
         [4.9000, 3.0000, 1.4000, 0.2000],
         [4.7000, 3.2000, 1.3000, 0.2000],
         [4.6000, 3.1000, 1.5000, 0.2000],
         [5.0000, 3.6000, 1.4000, 0.2000],
         [5.4000, 3.9000, 1.7000, 0.4000],
         [4.6000, 3.4000, 1.4000, 0.3000],
         [5.0000, 3.4000, 1.5000, 0.2000],
         [4.4000, 2.9000, 1.4000, 0.2000],
         [4.9000, 3.1000, 1.5000, 0.1000],
         [5.4000, 3.7000, 1.5000, 0.2000],
         [4.8000, 3.4000, 1.6000, 0.2000],
         [4.8000, 3.0000, 1.4000, 0.1000],
         [4.3000, 3.0000, 1.1000, 0.1000],
         [5.8000, 4.0000, 1.2000, 0.2000],
         [5.7000, 4.4000, 1.5000, 0.4000],
         [5.4000, 3.9000, 1.3000, 0.4000],
         [5.1000, 3.5000, 1.4000, 0.3000],
         [5.7000, 3.8000, 1.7000, 0.3000],
         [5.1000, 3.8000, 1.5000, 0.3000],
         [5.4000, 3.4000, 1.7000, 0.2000],
         [5.1000, 3.7000, 1.5000, 0.4000],
         [4.6000, 3.6000, 1.0000, 0.2000],
         [5

In [237]:
#TensorDataset: 入力データと教師データがセットのもの（セットにする関数）
dataset = torch.utils.data.TensorDataset(x, t)
dataset

In [238]:
dataset[0]

(tensor([5.1000, 3.5000, 1.4000, 0.2000]), tensor(0))

In [239]:
n_train = int(len(dataset) * 0.6)
n_val = int(len(dataset) * 0.2)
n_test = len(dataset) - n_train - n_val

In [240]:
print(n_train, n_val, n_test)

90 30 30


In [241]:
torch.manual_seed(0)
# torch.utils.data.random_split(dataset, lengths, generator=<torch._C.Generator object>)
train, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])

In [242]:
batch_size = 10

In [243]:
# DataLoader(): datasetからミニバッチを作成する関数
# エポックが回るごとに各ミニバッチを構成するデータがランダムに入れ替わる
train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val, batch_size)
test_loader = torch.utils.data.DataLoader(test, batch_size)

In [244]:
class Net(nn.Module):

    # 使用するオブジェクトを定義
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(4, 4)
        self.fc2 = nn.Linear(4, 3)

    # 順伝播
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

In [245]:
torch.manual_seed(0)

In [246]:
net = Net()

In [247]:
net

Net(
  (fc1): Linear(in_features=4, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=3, bias=True)
)

In [248]:
criterion = F.cross_entropy
criterion

<function torch.nn.functional.cross_entropy(input: torch.Tensor, target: torch.Tensor, weight: Union[torch.Tensor, NoneType] = None, size_average: Union[bool, NoneType] = None, ignore_index: int = -100, reduce: Union[bool, NoneType] = None, reduction: str = 'mean', label_smoothing: float = 0.0) -> torch.Tensor>

In [249]:
# net.parameters(): net内のパラメータを取得
for parameter in iter(net.parameters()):
    print("hey", parameter)

hey Parameter containing:
tensor([[-0.0037,  0.2682, -0.4115, -0.3680],
        [-0.1926,  0.1341, -0.0099,  0.3964],
        [-0.0444,  0.1323, -0.1511, -0.0983],
        [-0.4777, -0.3311, -0.2061,  0.0185]], requires_grad=True)
hey Parameter containing:
tensor([ 0.1977,  0.3000, -0.3390, -0.2177], requires_grad=True)
hey Parameter containing:
tensor([[ 0.1816,  0.4152, -0.1029,  0.3742],
        [-0.0806,  0.0529,  0.4527, -0.4638],
        [-0.3148, -0.1266, -0.1949,  0.4320]], requires_grad=True)
hey Parameter containing:
tensor([-0.3241, -0.2302, -0.3493], requires_grad=True)


In [250]:
# 最適化手法の選択
optimizer = torch.optim.SGD(net.parameters(), lr=0.1)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    lr: 0.1
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)

In [251]:
# バッチサイズ分のサンプルの抽出
batch = next(iter(train_loader))
batch

[tensor([[5.4000, 3.9000, 1.7000, 0.4000],
         [4.6000, 3.6000, 1.0000, 0.2000],
         [6.5000, 3.0000, 5.5000, 1.8000],
         [6.9000, 3.1000, 5.4000, 2.1000],
         [6.3000, 2.5000, 4.9000, 1.5000],
         [7.1000, 3.0000, 5.9000, 2.1000],
         [5.8000, 2.7000, 4.1000, 1.0000],
         [7.0000, 3.2000, 4.7000, 1.4000],
         [6.7000, 3.0000, 5.0000, 1.7000],
         [7.2000, 3.6000, 6.1000, 2.5000]]),
 tensor([0, 0, 2, 2, 1, 2, 1, 1, 1, 2])]

In [252]:
x, t = batch

In [253]:
y = net(x)
y

tensor([[-0.2557, -0.2605, -0.4679],
        [-0.2041, -0.2834, -0.5574],
        [-0.2786, -0.2244, -0.3632],
        [-0.2552, -0.2214, -0.3703],
        [-0.3241, -0.2302, -0.3493],
        [-0.2788, -0.2244, -0.3631],
        [-0.3241, -0.2302, -0.3493],
        [-0.3241, -0.2302, -0.3493],
        [-0.3090, -0.2282, -0.3539],
        [-0.1884, -0.2129, -0.3907]], grad_fn=<AddmmBackward0>)

In [254]:
loss = criterion(y, t)
loss

tensor(1.0882, grad_fn=<NllLossBackward0>)

In [255]:
net.fc1.weight

Parameter containing:
tensor([[-0.0037,  0.2682, -0.4115, -0.3680],
        [-0.1926,  0.1341, -0.0099,  0.3964],
        [-0.0444,  0.1323, -0.1511, -0.0983],
        [-0.4777, -0.3311, -0.2061,  0.0185]], requires_grad=True)

In [256]:
net.fc1.weight.grad

In [257]:
loss.backward()

In [258]:
net.fc1.weight.grad

tensor([[-0.2311, -0.1731, -0.0627, -0.0139],
        [ 0.7327,  0.3358,  0.6025,  0.2229],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000]])

In [259]:
optimizer.step()

In [260]:
net.fc1.weight

Parameter containing:
tensor([[ 0.0194,  0.2855, -0.4053, -0.3666],
        [-0.2659,  0.1005, -0.0702,  0.3742],
        [-0.0444,  0.1323, -0.1511, -0.0983],
        [-0.4777, -0.3311, -0.2061,  0.0185]], requires_grad=True)

In [261]:
max_epoch = 10

In [262]:
torch.manual_seed(0)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
net = Net().to(device)
device

device(type='cpu')

In [263]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.1)

In [264]:
i = 0
for batch in train_loader:
    print(i, batch)
    i += 1

0 [tensor([[5.4000, 3.9000, 1.7000, 0.4000],
        [4.6000, 3.6000, 1.0000, 0.2000],
        [6.5000, 3.0000, 5.5000, 1.8000],
        [6.9000, 3.1000, 5.4000, 2.1000],
        [6.3000, 2.5000, 4.9000, 1.5000],
        [7.1000, 3.0000, 5.9000, 2.1000],
        [5.8000, 2.7000, 4.1000, 1.0000],
        [7.0000, 3.2000, 4.7000, 1.4000],
        [6.7000, 3.0000, 5.0000, 1.7000],
        [7.2000, 3.6000, 6.1000, 2.5000]]), tensor([0, 0, 2, 2, 1, 2, 1, 1, 1, 2])]
1 [tensor([[6.9000, 3.2000, 5.7000, 2.3000],
        [5.0000, 3.5000, 1.6000, 0.6000],
        [6.9000, 3.1000, 4.9000, 1.5000],
        [5.7000, 3.8000, 1.7000, 0.3000],
        [4.4000, 2.9000, 1.4000, 0.2000],
        [6.0000, 3.0000, 4.8000, 1.8000],
        [5.1000, 3.8000, 1.9000, 0.4000],
        [6.6000, 3.0000, 4.4000, 1.4000],
        [4.7000, 3.2000, 1.3000, 0.2000],
        [5.7000, 2.8000, 4.1000, 1.3000]]), tensor([2, 0, 1, 0, 0, 2, 0, 1, 0, 1])]
2 [tensor([[5.5000, 3.5000, 1.3000, 0.2000],
        [7.9000, 3.8000, 

In [265]:
i = 0
# 学習ループ
for epoch in range(max_epoch):

    print(f"epoch: {i}")
    i += 1
    for batch in train_loader:

        # バッチサイズ分のサンプルを抽出
        x, t = batch

        # 学習時に使用するデバイスへデータの転送
        x = x.to(device)
        t = t.to(device)

        # パラメータの勾配を初期化
        optimizer.zero_grad()

        # 予測値の算出
        y = net(x)

        # 目標値と予測値から目的関数の値を算出
        loss = criterion(y, t)

        # 目的関数の値を表示して確認
        # item(): tensot.Tensor => float
        print('loss: ', loss.item())

        # 各パラメータの勾配を算出
        loss.backward()

        # 勾配の情報を用いたパラメータの更新
        optimizer.step()

epoch: 0
loss:  1.0581324100494385
loss:  1.0727126598358154
loss:  1.0202516317367554
loss:  1.0485258102416992
loss:  1.0162084102630615
loss:  0.9349002838134766
loss:  0.9057324528694153
loss:  0.8178885579109192
loss:  1.0845533609390259
epoch: 1
loss:  1.0116767883300781
loss:  0.857082188129425
loss:  0.8416250348091125
loss:  0.7966195940971375
loss:  0.7857252359390259
loss:  0.922939658164978
loss:  0.9749234914779663
loss:  0.8454588055610657
loss:  0.8937383890151978
epoch: 2
loss:  0.6872373819351196
loss:  0.8026294708251953
loss:  0.7394832372665405
loss:  0.8636043667793274
loss:  0.9335452914237976
loss:  0.88116854429245
loss:  0.8509734272956848
loss:  0.7113741636276245
loss:  0.6560307741165161
epoch: 3
loss:  0.7626632452011108
loss:  0.8948764801025391
loss:  0.6187215447425842
loss:  0.7431216239929199
loss:  0.8829702138900757
loss:  0.7989763021469116
loss:  0.8024595975875854
loss:  0.6409597396850586
loss:  0.4654276967048645
epoch: 4
loss:  0.75339353084564

In [266]:
y_label = torch.argmax(y, dim=1)
y_label

tensor([0, 0, 2, 0, 2, 2, 2, 2, 2, 2])

In [267]:
t

tensor([0, 0, 1, 0, 2, 1, 2, 2, 1, 2])

In [268]:
y_label == t

tensor([ True,  True, False,  True,  True, False,  True,  True, False,  True])

In [269]:
# モデルの初期化
torch.manual_seed(0)

# モデルのインスタンス化とデバイスへの転送
net = Net().to(device)

# 最適化手法の選択
optimizer = torch.optim.SGD(net.parameters(), lr=0.1)

In [270]:
for epoch in range(max_epoch):

    for batch in train_loader:

        x, t = batch
        x = x.to(device)
        t = t.to(device)
        optimizer.zero_grad()
        y = net(x)
        loss = criterion(y, t)

        # New：正解率の算出
        y_label = torch.argmax(y, dim=1)
        acc  = torch.sum(y_label == t) * 1.0 / len(t)
        print('accuracy:', acc)

        loss.backward()
        optimizer.step()

accuracy: tensor(0.6000)
accuracy: tensor(0.8000)
accuracy: tensor(0.6000)
accuracy: tensor(0.6000)
accuracy: tensor(0.6000)
accuracy: tensor(0.5000)
accuracy: tensor(0.4000)
accuracy: tensor(0.2000)
accuracy: tensor(0.4000)
accuracy: tensor(0.7000)
accuracy: tensor(0.2000)
accuracy: tensor(0.8000)
accuracy: tensor(0.5000)
accuracy: tensor(0.6000)
accuracy: tensor(0.6000)
accuracy: tensor(0.4000)
accuracy: tensor(0.8000)
accuracy: tensor(0.2000)
accuracy: tensor(0.5000)
accuracy: tensor(1.)
accuracy: tensor(0.7000)
accuracy: tensor(0.9000)
accuracy: tensor(0.7000)
accuracy: tensor(0.6000)
accuracy: tensor(0.5000)
accuracy: tensor(0.7000)
accuracy: tensor(0.7000)
accuracy: tensor(0.8000)
accuracy: tensor(0.6000)
accuracy: tensor(1.)
accuracy: tensor(0.6000)
accuracy: tensor(0.7000)
accuracy: tensor(0.4000)
accuracy: tensor(0.8000)
accuracy: tensor(0.6000)
accuracy: tensor(0.9000)
accuracy: tensor(0.7000)
accuracy: tensor(0.7000)
accuracy: tensor(0.7000)
accuracy: tensor(0.9000)
accuracy